In [8]:
!pip install streamlit
import streamlit as st
import pandas as pd
import numpy as np

st.set_page_config(page_title="Multi-Layer Process Log Engine")
st.title("Multi-Layer Process Log Engineering Framework")

uploaded_file = st.file_uploader(
    "Upload CSV or Excel file",
    type=["csv", "xlsx", "xls"]
)

if uploaded_file is not None:

    # ==============================
    # HANDLE CSV FILE
    # ==============================
    if uploaded_file.name.endswith(".csv"):
        with st.spinner("Loading CSV file..."):
            raw_df = pd.read_csv(uploaded_file)
        st.success("CSV file loaded successfully")

    # ==============================
    # HANDLE EXCEL FILE (MULTI-SHEET)
    # ==============================
    else:
        excel_file = pd.ExcelFile(uploaded_file)
        sheet_names = excel_file.sheet_names

        selected_sheet = st.selectbox(
            "Select the sheet you want to process",
            sheet_names
        )

        if st.button("Load Selected Sheet"):

            with st.spinner("Loading selected sheet..."):
                raw_df = pd.read_excel(uploaded_file, sheet_name=selected_sheet)

            st.success(f"Loaded sheet: {selected_sheet}")

    # Stop here if raw_df not loaded yet
    if "raw_df" not in locals():
        st.stop()

    # ==============================
    # STANDARDIZE COLUMN NAMES
    # ==============================
    raw_df.columns = (
        raw_df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )

    # ==============================
    # REMOVE DUPLICATE INCIDENT
    # ==============================
    disposition_col = None
    for col in raw_df.columns:
        if "disposition" in col:
            disposition_col = col
            break

    if disposition_col:
        before = len(raw_df)

        raw_df = raw_df[
            ~raw_df[disposition_col]
            .astype(str)
            .str.strip()
            .str.lower()
            .str.contains("duplicate incident", na=False)
        ]

        removed = before - len(raw_df)
        st.info(f"Duplicate Incident rows removed: {removed}")

    # ==============================
    # CONTINUE YOUR CASE + EVENT LOG LOGIC BELOW
    # ==============================

    st.success("Data cleaned successfully. Continue processing...") 

2026-02-25 05:03:23.092 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 05:03:23.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [10]:
!pip install pandas numpy openpyxl ipywidgets